In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

import muon as mu
from muon import MuData
from muon import prot as pt

import time
import psutil
import os
import memory_profiler



path='../../datasets/Mouse_Thymus5/'
adata_omics1 = sc.read_h5ad(path + 'adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(path + 'adata_ADT.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()


sc.pp.filter_genes(adata_omics2, min_cells=10)
sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
  
adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
sc.pp.pca(adata_omics1_high,n_comps=30)
  
# Protein
pt.pp.clr(adata_omics2)
sc.pp.scale(adata_omics2)
sc.pp.pca(adata_omics2,n_comps=30)


mdata = MuData({'rna': adata_omics1, 'adt': adata_omics2})

mdata.obs=pd.concat([adata_omics1.obs,
                       pd.DataFrame(adata_omics1.obsm["spatial"], columns=["imagerow", "imagecol"], index=adata_omics1.obs_names),
                      ], axis=1) 

n_inducing = 1000
%time %memit mu.tl.mofa(mdata, smooth_covariate=["imagerow", "imagecol"],outfile=path+"mefisto.hdf5",smooth_kwargs={"sparseGP": True, "frac_inducing": n_inducing/mdata.n_obs,"start_opt": 10, "opt_freq": 10,},n_factors=30,use_float32=True, seed=2021,quiet=False,gpu_mode=True,gpu_device=2)

rna=mdata["rna"]
rna.obsm["X_mefisto"]=mdata.obsm["X_mofa"]

result_path=path.replace("datasets","results")
rna.write_h5ad(result_path+'/adata_mefisto.h5ad')

/home/ws6tg/anaconda3/envs/scvi/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:325: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm["X_pca"] = X_pca



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
use_float32 set to True: replacing float64 arrays by float32 arrays to speed up computations...

Loaded view='rna' group='group1' with N=19737 samples and D=23375 features...
Loaded view='adt' group='group1' with N=19737 samples and D=128 features...


Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the w